<h1 style="text-align: center;">Llama3 fine-tuning 2end attempt</h1>

# Libraries

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
colab_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', colab_path)
sys.path.insert(0,colab_path)


Mounted at /content/drive


In [2]:
!pip install -q "torch==2.2.2" tensorboard

!pip install -q --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 73.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.2.2 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.4 MB/

Accessing Llama3 needs permission from Meta and needs to be requested, after receiving the permission, login to HuggingFace is required to use the access token.

In [ ]:
#Login to huggingface to access the model
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


# Dataset

This data has been annotated manually, as a part of a sentence might be worth highlighting and not all of it, we assigned 1 to the whole sentence indicating that the sentence is worth highlighting.
The test data set is the same one used for the OPEN AI API notebook.

In [ ]:
#Import sentence dataframe
df = pd.read_csv('/content/drive/MyDrive/Final Project/Field Project/Highlighted Papers/Sentence_dataframe.csv', index_col=False)


In [ ]:
#Train_test split (same used for OPEN AI API notebook)
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True, stratify=df['Highlight'])
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=42, shuffle=True, stratify=X_train['Highlight'] )
print(X_train.shape, X_test.shape, X_val.shape)


(2200, 6) (688, 6) (550, 6)


In [ ]:
X_train.head()


,Sentence,File_Name,Parse_Tree,Processed_Tree,Chunks,Highlight
599,The resistance values of the WO3 films on Ni f...,cleaned_Fulltext_Electrochromic solar water sp...,(ROOT (S (NP (NP (DT The) (NN resistance) (NNS...,(S (NP (NP (DT The) (NN resistance) (NNS value...,The resistance values of the WO3 films on Ni f...,0
2044,"Therefore, the present study showed that lower...",cleaned_Fulltext_School grade and sex_1.TXT,"(ROOT (S (ADVP (RB Therefore)) (, ,) (NP (DT t...","(S (ADVP (RB Therefore)) (, ,) (NP (DT the) (J...","Therefore,the present study showed that",0
1629,Two notable exceptions are the pyrethrins and ...,cleaned_Fulltext_Human health implications_1.TXT,(ROOT (S (NP (CD Two) (JJ notable) (NNS except...,(S (NP (CD Two) (JJ notable) (NNS exceptions))...,Two notable exceptions are the pyrethrins and ...,0
3268,"ECB, and participated in the study design and ...",cleaned_Fulltext_Standardisation of disk_1.txt,"(ROOT (S (NP (NNP ECB)) (, ,) (CC and) (VP (VP...","(S (NP (NNP ECB)) (, ,) (CC and) (VP (VP (VBD ...","ECB,and participated in the study design and c...",0
1722,"For example, ascorbic acid has received most a...",cleaned_Fulltext_Human health implications_1.TXT,"(ROOT (S (PP (IN For) (NP (NN example))) (, ,)...","(S (PP (IN For) (NP (NN example))) (, ,) (NP (...","For example,ascorbic acid has received most at...",0


In [ ]:
columns_to_drop = ['File_Name', 'Parse_Tree', 'Processed_Tree', 'Chunks']


In [ ]:
from datasets import Dataset, DatasetDict
# pandas to Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(X_train.drop(columns_to_drop,axis=1).reset_index(drop=True))
dataset_val = Dataset.from_pandas(X_val.drop(columns_to_drop,axis=1).reset_index(drop=True))
dataset_test = Dataset.from_pandas(X_test.drop(columns_to_drop,axis=1).reset_index(drop=True))


In [ ]:
# Combine into a DatasetDict (HuggingFace dataset format)
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 2200
    })
    val: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 550
    })
    test: Dataset({
        features: ['Sentence', 'Highlight'],
        num_rows: 688
    })
})

In [ ]:
#label distribution
X_train.Highlight.value_counts(normalize=True)


Highlight
0    0.565909
1    0.434091
Name: proportion, dtype: float64

# Model Configuration

We used Llama3 8B for finetuning, as the 70B caused severe memory issues even with Google Colab's L4 GPU with High RAM.

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B" #Model we used (Other options is Meta-Llama-3-70B)

In [ ]:
#quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, #4-bit precision to reduce memory
    bnb_4bit_quant_type = 'nf4', #non-linear quantization
    bnb_4bit_use_double_quant = True, #double quantization (to imrove performance)
    bnb_4bit_compute_dtype = torch.bfloat16
)


In a nutshell, "Low-Rank Adaptation (`LoRA`) is a technique designed to refine and optimise large language models based on". [Deeper Insights](https://deeperinsights.com/ai-blog/harnessing-the-power-of-lora-in-large-language-models-a-deep-dive-into-the-future-of-ai)

In [ ]:
#LoRA
lora_config = LoraConfig(
    r = 64, # dimension
    lora_alpha = 16,
    target_modules = [
        'q_proj', 'k_proj', 'v_proj', 'o_proj',  # Self-attention projections
        'gate_proj', 'up_proj', 'down_proj'     # MLP projections
    ], #all linear layers
    lora_dropout = 0.1,
    bias = 'none',
    task_type = 'SEQ_CLS' #sequence classification
)

Quantization of LoRA is usually called QLoRA. Based on [TextMine](https://textmine.com/post/an-introduction-to-llm-quantization) LLM quantization is an approach to processing large amounts of language data, which converts a continuous data set to a discrete data set – ultimately minimising the number of bits needed to display the signal.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2 #As it is a binary classification
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model = get_peft_model(model, lora_config) #Parameter-Efficient Fine-Tuning, applying Low-Rank Adaptation (LoRa)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
         

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id # "eos" stands for end-of-sequence
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id #match model's- and Tokenizer's pad_token_id
model.config.use_cache = False # True can cause memory issues
model.config.pretraining_tp = 1 #tensor parallelism (1 = single-device training)

In [ ]:
sentences = X_test.Sentence.tolist()

batch_size = 16

all_outputs = []

for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i + batch_size]

    # Tokenization
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Move tensors
    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    # store logits
    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])



In [ ]:
MAX_LEN = 256 #Max length of each input

def llama_preprocessing_function(examples):
    return tokenizer(examples['Sentence'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("Highlight", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer) #Padding


In [ ]:
#Metrics
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1_score': f1_score(labels, predictions, average='weighted')
    }


In [ ]:
#Training Arguments
training_args = TrainingArguments(
    output_dir = 'binary_classification',
    learning_rate = 2e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4, #accumulate gradients to avoid memory issues
    gradient_checkpointing=True,
    num_train_epochs = 5,
    weight_decay = 0.01,
    logging_steps=100, #Number of update steps between two logs
    #max_steps=1000,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [ ]:
#Finding optimal number of warmup steps
num_training_steps = len(tokenized_datasets['train']) // training_args.train_batch_size * training_args.num_train_epochs
num_warmup_steps = int(0.1 * num_training_steps)
num_warmup_steps

1100

In [ ]:
from transformers import Trainer, EarlyStoppingCallback, get_cosine_schedule_with_warmup
import torch

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# learning rate scheduler (Cosine)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=1100,
    num_training_steps=len(tokenized_datasets['train']) // training_args.train_batch_size * training_args.num_train_epochs
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=collate_fn,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], #Early stopping
    compute_metrics=compute_metrics,
)

# Training and result

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.848400,0.781176,0.618182,0.616358


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.848400,0.781176,0.618182,0.616358
2,0.722600,0.837026,0.654545,0.653136
3,0.550600,1.015375,0.667273,0.663939


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
# Prediction function
def make_predictions(model,df_test):

  sentences = X_test.Sentence.tolist()

  batch_size = 16

  all_outputs = []

  for i in range(0, len(sentences), batch_size):
      batch_sentences = sentences[i:i + batch_size]

      # Tokenization
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=256)

      # Move tensors
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      #store logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  X_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


make_predictions(model,X_test)

In [ ]:
# getting F1-score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

def get_performance_metrics(df):
    y_test = df.Highlight
    y_pred = df.predictions

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    print("F-1 Score:", f1_score(y_test, y_pred, average='weighted'))



In [ ]:
get_performance_metrics(X_test)

Confusion Matrix:
[[250 139]
 [138 161]]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.64      0.64       389
           1       0.54      0.54      0.54       299

    accuracy                           0.60       688
   macro avg       0.59      0.59      0.59       688
weighted avg       0.60      0.60      0.60       688

Accuracy Score: 0.5973837209302325
F-1 Score: 0.5974607107842733


In [1]:
import numpy as np

# Confusion matrix
confusion_matrix = np.array([
    [250 , 139],
    [138 , 161]
])

# number of classes
num_classes = confusion_matrix.shape[0]

# true positives
TP = np.diag(confusion_matrix)

# false positives
FP = np.sum(confusion_matrix, axis=0) - TP

# false negatives
FN = np.sum(confusion_matrix, axis=1) - TP

# epsilon to prevent division by zero
epsilon = 1e-10

# Precision, Recall, and F1-score
precision = TP / (TP + FP + epsilon)
recall = TP / (TP + FN + epsilon)
f1_score = 2 * (precision * recall) / (precision + recall + epsilon)

# Support
support = np.sum(confusion_matrix, axis=1)

#Weighted F1-score
weighted_f1_score = np.sum(f1_score * support) / np.sum(support)

#macro F1-score
macro_f1_score = np.mean(f1_score)

#micro F1-score
TP_micro = np.sum(TP)
FP_micro = np.sum(FP)
FN_micro = np.sum(FN)

precision_micro = TP_micro / (TP_micro + FP_micro + epsilon)
recall_micro = TP_micro / (TP_micro + FN_micro + epsilon)
micro_f1_score = 2 * (precision_micro * recall_micro) / (precision_micro + recall_micro + epsilon)

# Output the results
print("Weighted F1-score:", weighted_f1_score)
print("Macro F1-score:", macro_f1_score)
print("Micro F1-score:", micro_f1_score)


Weighted F1-score: 0.5974607107341018
Macro F1-score: 0.5905316238704331
Micro F1-score: 0.5973837208801457


We believe the F-1 score is a better metric in our case for our evaluation that represents both precision and recall of our model.

In [ ]:
metrics = train_result.metrics
max_train_samples = len(X_train)
metrics["train_samples"] = min(max_train_samples, len(X_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  8650684GF
  train_loss               =     0.8474
  train_runtime            = 2:22:12.54
  train_samples            =       2200
  train_samples_per_second =      1.289
  train_steps_per_second   =      0.322


In [ ]:
'/content/drive/My Drive/Llama3/att2'

In [ ]:
trainer.save_model("/content/drive/My Drive/Llama3/att2/Llama3_2att_trainer") #save trainer

In [ ]:
model.save_pretrained("/content/drive/My Drive/Llama3/att2/Llama3_2att_model.pt") #save model

In [ ]:
#load model
#model = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/Llama3/att2/Llama3_2att_model.pt")

In [3]:
!pip freeze > requirements.txt